In [1]:
### Load packages
using LinearAlgebra, Statistics, Compat, HypothesisTests # Statistical analysis
using RDatasets, MultivariateStats # Principal Component Analysis
using Distributions, GLM # OLS
using DataFrames, CSV # Data analysis
using Dates, Plots #printf # Misc...

In [2]:
### Supportive function -- part I
function blkdig(vararg)
    stop_row = 0
    stop_col = 0
    column = 0
    row = 0
    for i in 1:length(vararg)
        column = column + size(vararg[i],2)
        row = row + size(vararg[i],1)
    end
    
    blk = zeros(row, column)
    
    for i in 1:length(vararg)
        mat_comp = vararg[i]
        for j in 1:size(mat_comp, 2)
            for k in 1:size(mat_comp, 1)
                blk[stop_row + k, stop_col + j] = mat_comp[k,j]
            end
        end
        stop_row = stop_row + size(mat_comp,1)
        stop_col = stop_col + size(mat_comp,2)
    end
    
    return blk
end

### Supportive function -- part II
blocks = [1 0 0 1 0
            1 0 1 0 0
            1 0 0 1 0]
function sort_vec(blocks)
    uniq = [ blocks[1,:]]
    
    for i in 2:size(blocks,1)
        sum = 0
        for j in 1:size(uniq,1)
            sum = sum + (blocks[i,:] == uniq[j])
        end
        if sum == 0
            push!(uniq, blocks[i,:])
        end
    end
    return uniq
end

function ismember(blocks, str)
    idx = []
    for i in 1:size(blocks,1)
        if blocks[i,:] == str
            push!(idx, i)
        end
    end
    return idx
end

sort_vec(blocks)

2-element Array{Array{Int64,1},1}:
 [1, 0, 0, 1, 0]
 [1, 0, 1, 0, 0]

In [3]:
struct Spec
    r::Int
    p::Int
    nQ::Int
    blocks::AbstractMatrix{Float64}
end

struct inits
    A::AbstractMatrix{Float64}
    G::AbstractMatrix{Float64}
    Q::AbstractMatrix{Float64}
    R::AbstractMatrix{Float64}
    Z::AbstractMatrix{Float64}
    V::AbstractMatrix{Float64}
end

struct Sm
    Zm::AbstractMatrix{Float64}
    ZmU::AbstractMatrix{Float64}
    Vm::AbstractArray{Float64,3}
    VmU::AbstractArray{Float64,3}
    loglik::Float64
    k_t::AbstractMatrix{Float64}
end

struct Sm1
    ZmT::AbstractMatrix{Float64}
    VmT::AbstractArray{Float64,3}
    VmT_1::AbstractArray{Float64,3}
end

In [4]:
##### Creating Dynamic Factor Model (DFM) to forecast GDP: 
# y_t - μ = G_θ * x_t
# x_{t+1} = A_θ * x_t + η_t
function dfm(X, Spec, inits, threshold = 1e-5)
    ### Set up
    #printf("Estimating the dynamic factor model (DFM) ... \n\n")
    
    (T, N) = size(X)
    # Identify spec 
    r = Spec.r
    p = Spec.p
    nQ = Spec.nQ
    blocks = Spec.blocks
    
    i_idio = Bool.([ones(N-nQ,1); zeros(nQ,1)])
    
    # We should not have constraint
    R_mat = [2 -1 0 0 0
             3 0 -1 0 0
             2 0 0 -1 0
             1 0 0 0 -1]
    q = zeros(4,1)
    
    max_iter = 5000
    
    ### Prepare data
    #Mx = mean(X, "omitnan")
    #Wx = std(X, "omitnan")
    #xMissing = (X - repmat(Mx, T, 1))./repmat(Wx, T,1) #? Standardize series???
    
    ### Initial Condition
    A = inits.A
    G = inits.G
    Q = inits.Q
    R = inits.R
    Z_0 = inits.Z
    V_0 = inits.V
    
    previous_loglik = -Inf
    num_iter = 0
    LL = -Inf
    converged = false
    
    # ? y for the estimation is WITH missing data
    
    ### EM loop
    #? remove the leading and ending nans
    # y_est
    println("run EM loop")
    while (num_iter < max_iter) & !converged
        (G_new, R_new, A_new, Q_new, Z_0, V_0, loglik ) = EMstep(X, A, G, Q, R, Z_0, V_0, r, p, R_mat, q, nQ, i_idio, blocks)
        G = G_new
        R = R_new
        A = A_new
        Q = Q_new
        
        if num_iter > 2 
            # change from decrease(num_iter+1)
            converged = em_converged(loglik, previous_loglik, threshold, check_decreased = 1)[1] #####?????
        end
        
        if (mod(num_iter,10) == 0) && (num_iter > 0)
            ##### Display something
        end
        
        LL = [LL loglik]
        previous_loglik = loglik
        num_iter = num_iter + 1
    end
    
    ##### Display something
    
    ### Final run of KF
    Zsmooth = runKF(X, A, G, Q, R, Z_0, V_0)[1] # use X with missing data
    x_sm = G*Zsmooth[2:end,:] # originally Zsm*G'
    
    ##### Display something
    
    return x_sm

end

dfm (generic function with 2 methods)

In [5]:
### Y_t = G_t X_t + e_t for e_t ~ N(0, R)
### X_t = A X_{t-1} + mu_t for mu_t ~ N(0,Q)
function EMstep(y,A,G,Q,R,x_0, σ_0, r, p, R_mat,q, nQ, i_idio, blocks)
    
    (n,T) = size(y)
    nM = n - nQ
    pC = size(R_mat,2)
    ppc = max(p, pC)
    num_blocks = size(blocks, 2) # how many columns are there?
    
    
    (x_sm, σ_sm, vv_sm, loglik) = runKF(y, A, G, Q, R, x_0, σ_0)
    
    print("KFrun succeed")
    A_new = A
    Q_new = Q
    σ_0_new = σ_0
    
    ##### Transition equation
    ### Update factor parameters individually
    for i = 1:num_blocks
        r_i = r[i]
        rp = r_i*p
        rp1 = sum(r[1:i-1])*ppC
        b_subset = rp1+1:rp1+rp # Subset blocks: Helps for subsetting x_sm, σ_sm
        t_start = rp1+1
        t_end = rp1+r_i*ppC
        
        EZZ = x_sm[b_subset, 2:end]*(x_sm[b_subset, 2:end])' + sum(σ_sm[b_subset, b_subset, 2:end], 3)
        EZZ_BB = x_sm[b_subset, 1:end-1]*(x_sm[b_subset, 1:end-1])' + sum(σ_sm[b_subset, b_subset, 1:end-1], 3)
        EZZ_FB = x_sm[b_subset, 2:end]*(x_sm[b_subset, 1:end-1])' + sum(vv_sm[b_subset, b_subset, :], 3)
        
        A_i = A[t_start:t_end, t_start:t_end]
        Q_i = Q[t_start:t_end, t_start:t_end]
        
        A_i[1:r_i,1:rp] = EZZ_FB[1:r_i,1:rp]*inv(EZZ_BB[1:r_i,1:rp])
        Q_i[1:r_i,1:r_i] = (EZZ[1:r_i,1:r_i] - A_i[1:r_i,1:rp]*EZZ_FB[1:r_i,1:rp]') / T
        
        A_new[t_start:t_end, t_start:t_end] = A_i
        Q_new[t_start:t_end, t_start:t_end] = Q_i
        σ_0_new[t_start:t_end, t_start:t_end] = σ_sm[t_start:t_end, t_start:t_end, 1]
    end
    
    ### Update parameters for idiosyncratic component
    rp1 = sum(r)*ppC
    niM = sum(i_idio[1:nM])
    t_start = rp1+1
    i_subset = t_start:rp1+niM
    
    EZZ = Diagonal(Diagonal(x_sm[t_start:end,2:end]*(x_sm[t_start:end, 2:end])'))+Diagonal(Diagonal(sum(σ_sm[t_start:end, t_start:end, 2:end], 3)))
    EZZ_BB = Diagonal(Diagonal(x_sm[t_start:end,1:end-1]*(x_sm[t_start:end, 1:end-1])'))+Diagonal(Diagonal(sum(σ_sm[t_start:end, t_start:end, 1:end-1], 3)))
    EZZ_FB = Diagonal(Diagonal(x_sm[t_start:end,2:end]*(x_sm[t_start:end, 1:end-1])'))+Diagonal(Diagonal(sum(vv_sm[t_start:end, t_start:end, :], 3)))
    
    A_i = EZZ_FB*Diagonal(inv(Diagonal((EZZ_BB))))
    Q_i = (EZZ - A_i*EZZ_FB') / T
    
    A_new[i_subset, i_subset] = A_i[1:niM,1:niM]
    Q_new[i_subset, i_subset] = Q_i[1:niM,1:niM]
    σ_0_new[i_subset, i_subset] = Diagonal(Diagonal(σ_sm[i_subset,i_subset,1]))
    
    ##### Observation equation
    ### Maximization step
    x_0 = x_sm[:,1]
    
    nanY = ismissing.(y)
    y[nanY] .= 0
    
    G_new = G
    
    bl = sort_vec(blocks) # list of unique blocks
    n_bl = size(bl,1) # number of unique blocks
    
    bl_idxM = []
    bl_idxQ = []
    R_con = []
    q_con = []
    
    for i in 1:num_blocks
        bl_idxQ = [bl_idxQ repeat(bl[:,i],1,r[i]*ppC)]
        bl_idxM = [bl_idxM repeat(bl[:,i],1,r[i]) zeros(n_bl, r[i]*(ppC-1))]
        R_con = blkdig(R_con, kron(R_mat, Matrix{Float64}(I,r[i],r[i])))
        q_con = [q_con zeros(r[i]*size(R_mat,1),1)]
    end
    
    # Indicator for m/q blocks in observation matrix
    bl_idxM = convert.(Bool, bl_idxM)
    bl_idxQ = convert.(Bool, bl_idxQ)
    
    i_idio_M = i_idio[1:nM]
    n_idio_M = length(find(i_idio_M))
    c_i_idio = cumsum(i_idio)
    
    for i in 1:n_bl # loop through unique loading
        bl_i = bl[i,:]
        rs = sum(convert.(Bool, bl_i)) # total num of blocks loaded (e.g [1 0 0 1] = 2)
        idx_i = ismember(blocks, bl_i) # indixes for [1 0 0 1]
        idx_iM = idx_i[idx_i < nM+1] # consider only monthly variables
        n_i = length(idx_iM)

        denom = zeros(n_i*rs, n_i*rs)
        nom = zeros(n_i, rs)
        
        i_idio_i = i_idio_M(idx_iM)
        i_idio_ii = c_i_idio(idx_iM)
        i_idio_ii = i_idio_ii(i_idio_i)
        
        ## update monthly variables
        for t in 1:T
            Wt = Diagonal(.!(nanY[idx_iM,t]))
            denom = denom + kron(x_sm[bl_idxM[i,:], t+1]*x_sm[bl_idxM[i,:], t+1]' 
                + σ_sm[bl_idx[i,:], bl_idxM[i, :], t+1], Wt)
            
            nom = nom + y[idx_iM,t]*x_sm[bl_idxM[i,:],t+1]' - Wt[:, i_idio_i]*(x_sm[rp1+i_idio_ii, t+1]*x_sm[bl_idxM[i,:],t+1]'
                + σ_sm[rp1+i_idio_ii, bl_idxM[i,:], t+1])
        end
        vec_G = inv(denom)*nom[:] #Eq13 in BGR 2010
        G_new[idx_iM, bl_idxM[i,:]] = reshape(vec_G, n_i, rs)
        
        ## update quarterly variables
        
        idx_iQ = idx_i[idx_i > nM] # consider only quarterly variables
        rps = rs*ppC
        R_con_i = R_con[:, bl_inxQ[i,:]]
        q_con_i = q_con
        no_c = .!(any(R_con_i,2))
        R_con_i[no_c, :] = []
        q_con_i[no_c, :] = []
        
        for j in idx_iQ'
            denom = zeros(rps,rps)
            nom = zeros(1, rps)
            
            idx_jQ = j - nM
            i_idio_jQ = (rp1+n_idio_M+5*(idx_jQ-1)+1:rp1+n_idio_M + 5*idx_jQ)
            
            V_0_new[i_idio_jQ, i_idio_jQ] = σ_sm[i_idio_jQ, i_idio_jQ, 1]
            A_new[i_idio_jQ[1], i+_idio_jQ[1]] = A_i[i_idio_jQ[1]-rp1, i_idio_jQ[1]-rp1]
            Q_new[i_idio_jQ[1], i_idio_jQ[1]] = Q_i[i_idio_jQ[1]-rp1, i_idio_jQ[1]-rp1]
            
            for t in 1:T
                Wt = Diagonal(.!(nanY[k,t]))
                denom = denom+kron(x_sm[bl_idxQ[i,:], t+1]*x_sm[bl_idxQ[i,:], t+1]' + σ_sm[bl_idxQ[i,:], bl_idxQ[i,:], t+1], Wt)
                nom = nom +y(j,t)*x_sm[bl_idxQ[i,:], t+1]'
                nom = nom - Wt*([1 2 3 2 1]*x_sm[i_idio+jQ,t+1]*x_sm[bl_idxQ[i,:],t+1]'+[1 2 3 2 1]*σ_sm[i_idio_jQ, bl_idxQ[i,:],t+1])
            end
            
            G_i = inv(denom)*nom'
            #BGR eq13
            G_i_constr = G_i - inv(denom)*R_con_i'*inv(R_con_i*inv(denom)*R_con_i')*(R_con_i*G_i -q_con_i)
            G_new[j,bl_idxQ[i,:]] = G_i_constr
        end
    end
    
    ### Update covariance of residuals for observation equation
    R_new = zeros(n,n)
    for t in 1:T
        Wt = Diagonal(.!(nanY[:,t]))
        R_new = R_new + (y[:,t] - Wt*G_new*x_sm[:,t+1])*(y[:,t]-Wt*G_new*x_sm[:,t+1])'+Wt*G_new*σ_sm[:,:,t+1]*G_new'*Wt + (Matrix{Float64}(I,n,n)-Wt)*R*(Matrix{Float64}(I,n,n)-Wt)
    end
    
    R_new = R_new / T
    RR = Diagonal(R_new)
    RR[i_idio+_M] = 1e-04
    RR[nM+1:end] = 1e-04
    R_new = Diagonal(RR)
    
    return G_new, R_new, A_new, Q_new, x_0, σ_0, loglik
end

EMstep (generic function with 1 method)

In [88]:
### Y_t = C_t Z_t + e_t for e_t ~ N(0, R)
### Z_t = A Z_{t-1} + mu_t for mu_t ~ N(0,Q)
function runKF(Y, A, C, Q, R, Z_0, V_0)
    S = SKF(Y,A,C,Q,R,Z_0,V_0)
    S, S1 = FIS(A,S)
    
    zsmooth = S1.ZmT
    Vsmooth = S1.VmT
    VVsmooth = S1.VmT_1
    loglik = S.loglik
    
    return zsmooth, Vsmooth, VVsmooth, loglik
end

function SKF(Y, A, C, Q, R, Z_0, V_0)
    # Initialize output values
    m = size(C,2)
    nobs = size(Y,2) # number of observations
    S = Sm(zeros(m, nobs), zeros(m, nobs+1), zeros(m,m, nobs), zeros(m,m, nobs+1), 0, zeros(m,m))
    
    Zu = Z_0
    Vu = V_0
    
    S.ZmU[:,1] = Zu
    S.VmU[:,:,1] = Vu
    
    # Kalman filter procedure
    for t in 1:nobs
        Z = A*Zu
        V = A*Vu*A' +Q
        V = 0.5*(V+V')
        
        (Y_t, C_t, R_t) = MissData(Y[:,t], C, R)
        
        if isempty(Y_t)
            Zu = Z
            Vu = V
        else
            VC = V*C_t'
            temp = C_t*VC +R_t
            print(size(temp))
            iF = inv(temp)
            
            VCF = VC*iF
            innov = Y_t - C_t*Z
            Zu = Z + VCF*innov
            
            Vu = V - VCF*VC'
            Vu = 0.5*(Vu+Vu')
            
            println(temp)
            print(innov'*iF*innov)
            S.loglik = S.loglik + 0.5*(log(det(iF)) - (innov'*iF*innov))
            
            print(2)
        end
        
        S.Zm[:,t] = Z
        S.Vm[:,:,t] = V
        S.ZmU[:,t+1] = Zu
        S.VmU[:,:,t+1] = Vu
    end
    
    if isempty(Y_t)
        S.k_t = zeros(m,m)
    else
        S.k_t = VCF*C_t
    end
    print("Filtering success")
    return S
end

function FIS(A,S)
    (m, nobs) = size(S.Zm)
    S1 = Sm1(zeros(m, nobs+1), zeros(m,m,nobs+1), zeros(m,m,nobs))
    
    S1.ZmT[:, nobs+1] = squeeze(S.ZmU[:,nobs+1])
    S1.VmT[:,:,nobs+1] = squeeze(S.VmU[:,:,nobs+1])
    S1.VmT_1[:,:,nobs] = (Matrix{Float}(I, m, m)-S.k_t)*A*squeeze(S.VmU[:,:,nobs])
    
    J_2 = squeeze(S.VmU[:,:,nobs])*A'*pinv(squeeze(S.Vm[:,:,nobs]))
    
    # Run smoothing algorithm
    for t in nobs:-1:1
        VmU = squeeze(S.VmU[:,:,t])
        Vm1 = squeeze(S.Vm[:,:,t])
        V_T = squeeze(S1.VmT[:,:,t+1])
        V_T1 = squeeze(S1.VmT_1[:,:,t])
        J_1 = J_2
        
        # update smoothed factor estimate / covariance matrix
        S1.ZmT[:,t] = S.ZmU[:,t]+J_1*(S1.ZmT[:,t+1] - A*S.ZmU[:,t])
        S1.VmT[:,:,t] = VmU + J_1*(V_T - Vm1)*J_1'
        
        if t > 1
            J_2 = squeeze(S.VmU[:,:,t-1])*A'*pinv(squeeze(S.Vm[:,:,t-1]))
            S1.VmT_1[:,:,t-1] = VmU*J_2' + J_1*(V_T1 - A*VmU)*J_2'
        end
    end
    
    return S, S1
end

function MissData(y,C,R)
    ix = .!ismissing.(y)
    e = Matrix{Float64}(I,size(y,1),size(y,1))
    L = e[:,ix]
    y = y[ix]
    C = C[ix,:]
    R = R[ix,ix]
    
    return y, C, R
end

#function InitialCond -- PCA for prior information

function em_converged(loglik, previous_loglik, threshold = 1e-4, check_decreased = 1)
    converged = 0
    decrease = 0
    
    if check_decreased
        if loglik - previous_loglik < -1e-3
            printf(1, "******likelihood decreased from %6.4f to %6.4f!\n", previous_loglik, loglik)
            decrease = 1
        end
    end
    
    delta_loglik = abs(loglik - previous_loglik)
    avg_loglik = (abs(loglik) + abs(previous_loglik) + eps)/2
    
    if (delta_loglik / avg_loglik) < threshold
        converged = 1
    end
    
    return converged, decrease 
end

em_converged (generic function with 3 methods)

In [66]:
A_θ = convert(Matrix{Float64}, CSV.read("A_theta.csv", header = false))
G_θ = convert(Matrix{Float64}, CSV.read("G_theta.csv", header = false))
Q_θ = convert(Matrix{Float64}, CSV.read("Q_theta.csv", header = false))
R_θ = convert(Matrix{Float64}, CSV.read("R_theta.csv", header = false))
Z_θ = convert(Matrix{Float64}, CSV.read("Z_theta.csv", header = false))
V_θ = convert(Matrix{Float64}, CSV.read("V_theta.csv", header = false))
block = convert(Matrix{Float64}, CSV.read("block.csv", header = false))
X = CSV.read("X.csv", header = true)
X = convert(Matrix{Union{Missing,Float64}},X[2:end,])
print("Success")

Success

In [89]:
### Trial
Sp = Spec(1,1,2,block)
Inits = inits(A_θ, G_θ, Q_θ, R_θ, Z_θ, V_θ)

dfm(X', Sp, Inits)


run EM loop


(101, 101)[0.0319378 1.73864e-7 1.86427e-7 1.94234e-7 1.42099e-7 7.22821e-9 2.85303e-8 -2.96346e-7 1.51369e-7 -2.68491e-8 2.29715e-7 3.29734e-7 4.09408e-7 2.21856e-7 8.26103e-7 -4.71044e-8 -4.0161e-6 -5.49506e-8 1.15946e-7 -7.98901e-6 -6.66118e-6 -7.77577e-7 -9.57151e-7 -3.82318e-6 -2.72926e-6 -4.52922e-6 9.27679e-8 4.74062e-7 1.98698e-7 5.29503e-7 4.22031e-7 4.32696e-7 4.41312e-7 2.17105e-8 9.40219e-8 3.17916e-8 3.17647e-8 2.65883e-8 -9.85694e-8 -0.0602592 -5.84524e-6 -0.0626358 -9.63e-5 -0.0041385 -0.00147982 -0.00147397 -0.00171956 2.82531e-7 1.41252e-7 -6.29907e-8 2.09921e-7 -2.00348e-7 309.43 3.08131e-8 2.94674e-8 8.69168e-9 -4.16643e-11 3.03595e-7 2.82808e-6 4.23312e-6 5.75761e-6 2.62042e-6 2.40215e-6 2.33287e-6 8.92763e-6 0.000775534 0.000446982 -3.32205e-5 -4.87017e-5 0.000108162 0.000217073 0.000129485 8.54357e-8 8.78605e-8 1.7436e-7 -5.95755e-7 3.73665e-7 -6.60607e-7 4.32218e-8 -1.83781e-7 3.53626e-7 -1.18717e-7 8.66134e-8 -7.88182e-8 -1.94094e-8 2.59102e-8 8.16207e-8 5.42035

-7.21547e-9; -7.77577e-7 -9.63503e-7 -4.77964e-7 -5.07725e-7 -2.04583e-7 4.96397e-7 5.28831e-8 1.67357e-7 -1.82741e-7 1.3455e-6 -7.78863e-7 1.16924e-7 -1.8617e-6 -3.16248e-7 -2.08673e-6 6.83911e-7 -5.80989e-5 5.11829e-7 -1.23204e-6 3.63358e-5 1.41931e-5 0.264193 7.29808e-6 3.68483e-5 2.63658e-5 4.39588e-5 -8.14067e-7 -4.43406e-6 -3.709e-6 -5.50035e-6 -3.60756e-6 -3.78097e-6 -3.62838e-6 -1.64652e-7 -7.30821e-7 -8.90361e-8 -1.82238e-7 -2.04274e-7 7.58422e-7 1.23756 4.78215e-5 1.20131 -0.000451065 0.0117846 0.00201212 0.00545379 0.00558364 -2.24513e-6 -1.12246e-6 -5.08157e-7 -1.66814e-6 1.59206e-6 -2458.88 -2.44857e-7 -2.34163e-7 -6.90684e-8 1.46236e-10 -2.41251e-6 -2.24733e-5 -3.36385e-5 -4.57528e-5 -2.08232e-5 -1.90887e-5 -1.85381e-5 -7.09434e-5 -0.00616278 -0.00355194 0.000263987 0.000387008 -0.000859511 -0.00172497 -0.00102895 -2.00737e-7 -1.71723e-7 -5.09781e-7 2.63081e-6 -3.84619e-7 2.41925e-6 -1.32147e-7 6.10741e-7 -1.0505e-6 3.9345e-7 -2.36145e-7 2.88473e-7 5.69953e-8 -6.56731e-8 

-8 -2.92527e-7 1.59787e-8 -7.38487e-8 1.27023e-7 -4.75746e-8 2.85538e-8 -3.48812e-8 -6.89167e-9 7.94097e-9 2.93755e-8 2.00183e-8 -5.08701e-8 -1.11192e-8 -1.16718e-7 -3.77498e-8 9.6633e-10 -1.67088e-8 2.72517e-8 2.29662e-7 -1.91525e-8 1.76019e-8 3.62076e-319 4.73323e-8 1.01846e-10; 3.17916e-8 3.93932e-8 1.95418e-8 2.07585e-8 8.36445e-9 -2.02954e-8 -2.16215e-9 -6.84245e-9 7.47143e-9 -5.50114e-8 3.18442e-8 -4.78048e-9 7.61164e-8 1.29299e-8 8.53169e-8 -2.7962e-8 2.3754e-6 -1.06214e-8 7.77974e-9 -5.2077e-6 -5.89152e-6 -8.90361e-8 -3.40601e-7 -6.19879e-7 -4.36664e-7 -7.04955e-7 2.27423e-8 8.98532e-8 -1.40385e-7 4.76621e-8 1.12688e-7 1.07623e-7 1.31691e-7 7.81088e-9 3.21179e-8 0.594483 1.70725e-8 9.31286e-9 -3.44166e-8 0.0534205 -1.76706e-6 0.0473439 1.8442e-5 -0.000481819 -8.22664e-5 -0.00022298 -0.00022829 9.17931e-8 4.58922e-8 2.07762e-8 6.82025e-8 -6.50922e-8 100.532 1.00111e-8 9.57384e-9 2.82389e-9 -5.97892e-12 9.86366e-8 9.18832e-7 1.37532e-6 1.87062e-6 8.51365e-7 7.8045e-7 7.57939e-7 2

00339076 -36.3344 26.2515 4.11342 2.15053 1.07713 1.49702 0.000163893 8.19388e-5 0.000274846 0.000121773 -0.00011622 1.79497e5 1.78744e-5 1.70938e-5 5.04196e-6 3.28935e-8 0.000176112 0.00164054 0.00245559 0.00333993 0.00152008 0.00139347 0.00135327 0.00517883 0.449879 0.25929 -0.0192709 -0.0282514 0.0627439 0.125922 0.0751127 -9.80516e-5 -0.00011155 -0.000169203 0.000303709 -0.000581133 0.000490484 -4.01597e-5 0.000155683 -0.000338045 0.000100896 -8.9327e-5 5.85734e-5 1.87589e-5 -2.8256e-5 -7.78782e-5 -5.04153e-5 -3.07112e-5 -6.71288e-6 -7.04648e-5 -2.27902e-5 4.36995e-7 -1.10499e-5 1.72919e-5 0.000138651 -1.15627e-5 1.06266e-5 2.18591e-316 -0.000104503 3.99242e-6; -0.0041385 -0.00354731 -0.00501496 -0.00520371 -0.00417105 -0.00131799 -0.00104247 0.00926442 -0.00451993 -0.00206337 -0.005765 -0.0109696 -0.00924135 -0.00651903 -0.0222906 3.83706e-5 0.25727 0.000832808 -0.00175723 0.121078 0.100954 0.0117846 0.0145062 0.0579425 0.0413635 0.0686429 -0.00140595 -0.00718468 -0.00301138 -0.00

...

460115 -0.000785606 -0.00212936 -0.00218006 -3.23432e-5 -1.64363e-5 1.98403e-7 -2.95349e-5 1.96572e-5 960.316 -1.14063e-6 -1.49287e-6 -1.46794e-6 -5.70959e-11 0.37041 0.000493848 0.000283851 0.000172432 0.000176002 8.09058e-6 -0.000227774 -0.00107016 -0.0301518 -0.012361 0.000396235 -0.00343317 -0.01415 -0.0151417 -0.0133202 7.8375e-8 6.7047e-8 1.99037e-7 -1.02716e-6 1.50169e-7 -9.44562e-7 5.1595e-8 -2.38456e-7 4.10154e-7 -1.53617e-7 9.21997e-8 -1.1263e-7 -2.2253e-8 2.56412e-8 9.48527e-8 6.46387e-8 5.68935e-6 1.5212e-6 1.20184e-5 4.53599e-6 2.33726e-9 -5.91004e-8 9.24857e-8 -2.64987e-5 -6.18431e-8 5.6836e-8 1.16913e-318 1.52835e-7 3.28857e-10; 2.82808e-6 3.5043e-6 1.73838e-6 1.84662e-6 7.44076e-7 -1.80542e-6 -1.92338e-7 -6.08684e-7 6.64636e-7 -4.89364e-6 2.83276e-6 -4.25257e-7 6.77108e-6 1.15021e-6 7.58953e-6 -2.48741e-6 0.000211308 -1.58817e-6 3.35105e-6 -0.000230896 -0.00019252 -2.24733e-5 -2.76633e-5 -0.000110497 -7.88804e-5 -0.000130902 2.68115e-6 1.37012e-5 5.74271e-6 1.53036e-5 1

2033e-5 2.02491e-5 4.16529e-316 5.44508e-5 1.17163e-7; 0.000217073 0.000268977 0.000133431 0.00014174 5.71126e-5 -0.000138577 -1.47632e-5 -4.67204e-5 5.1015e-5 -0.000375618 0.000217432 -3.26412e-5 0.000519724 8.82859e-5 0.000582545 -0.000190925 0.0162192 -0.000121902 0.000257215 -0.0177228 -0.0147771 -0.00172497 -0.00212334 -0.00848132 -0.00605458 -0.0100476 0.000205796 0.00105166 0.00044079 0.00117465 0.000936231 0.00095989 0.000979003 4.81625e-5 0.000208578 7.05262e-5 7.04666e-5 5.89832e-5 -0.000218666 -133.679 -0.012967 -138.951 0.125922 -3.28987 -0.561716 -1.52251 -1.55876 0.00132802 0.000669566 0.00014186 0.00110291 -0.000872528 6.86431e5 9.44521e-5 9.88141e-5 5.08384e-5 -4.08241e-8 -0.0151417 -0.00396591 0.003676 0.00950976 0.00226943 0.00531546 0.0101362 0.0429804 2.40772 1.28179 -0.0842363 -0.0387564 0.54573 8.92469 0.576912 5.60389e-5 4.79393e-5 0.000142314 -0.000734432 0.000107373 -0.000675372 3.6891e-5 -0.000170498 0.000293265 -0.000109838 6.59237e-5 -8.0532e-5 -1.59112e-5 1

-8 -1.6766e-8 -2.33178e-8 -2.42005e-8 -1.93099e-8 -5.85742e-9 -4.78083e-9 4.27648e-8 -2.09081e-8 -8.92262e-9 -2.69051e-8 -5.0499e-8 -4.33961e-8 -3.01784e-8 -1.03627e-7 4.74542e-10 1.16025e-6 4.0278e-9 -8.4987e-9 5.85583e-7 4.88255e-7 5.69953e-8 7.01578e-8 2.80234e-7 2.00051e-7 3.31985e-7 -6.79975e-9 -3.47481e-8 -1.45643e-8 -3.88118e-8 -3.09343e-8 -3.1716e-8 -3.23475e-8 -1.59135e-9 -6.89167e-9 -2.33028e-9 -2.3283e-9 -1.94888e-9 7.22499e-9 0.00441691 4.28447e-7 0.00459112 1.87589e-5 0.000506336 0.000202232 0.000168249 0.000203774 -2.07091e-8 -1.03536e-8 1.43204e-8 -1.53869e-8 1.46852e-8 -22.6808 -2.25856e-9 -2.15992e-9 -6.37087e-10 4.83209e-12 -2.2253e-8 -2.07294e-7 -3.10282e-7 -4.22024e-7 -1.92073e-7 -1.76074e-7 -1.70996e-7 -6.54382e-7 -5.68455e-5 -3.27631e-5 2.43501e-6 3.56976e-6 -7.92814e-6 -1.59112e-5 -9.49103e-6 -1.08621e-8 -1.15043e-8 -2.12048e-8 6.39011e-8 -5.22526e-8 7.56472e-8 -5.20082e-9 2.16443e-8 -4.28466e-8 1.39918e-8 -1.06979e-8 9.02725e-9 0.223427 -3.24804e-9 -9.88489e-9 -

6.20157e-8 -3.3875e-9 1.56559e-8 -2.69289e-8 1.00858e-8 -6.05342e-9 7.39481e-9 1.46103e-9 -1.68349e-9 -6.2276e-9 -4.24389e-9 1.07845e-8 2.35728e-9 2.47442e-8 8.00296e-9 -1.53454e-10 3.88027e-9 -6.07219e-9 -4.86883e-8 0.264671 2.54608e-7 5.6403e-319 -1.00345e-8 -2.15913e-11; 1.83188e-8 2.2699e-8 1.12603e-8 1.19614e-8 4.81974e-9 -1.16945e-8 -1.24587e-9 -3.94274e-9 4.30516e-9 -3.16985e-8 1.83491e-8 -2.75459e-9 4.38595e-8 7.45045e-9 4.9161e-8 -1.61122e-8 1.36874e-6 -1.02873e-8 2.17064e-8 -1.49563e-6 -1.24704e-6 -1.45571e-7 -1.79189e-7 -7.1574e-7 -5.10946e-7 -8.47917e-7 1.73671e-8 8.87493e-8 3.71983e-8 9.91285e-8 7.90086e-8 8.10052e-8 8.26182e-8 4.06443e-9 1.76019e-8 5.95171e-9 5.94668e-9 4.9776e-9 -1.84532e-8 -0.0112811 -1.09429e-6 -0.0117261 1.06266e-5 -0.000277632 -4.74033e-5 -0.000128485 -0.000131544 5.28927e-8 2.64438e-8 1.19716e-8 3.92994e-8 -3.75072e-8 57.9285 5.76854e-9 5.51661e-9 1.62717e-9 -3.44515e-12 5.6836e-8 5.29446e-7 7.92485e-7 1.07788e-6 4.90571e-7 4.49708e-7 4.36737e-7 1.6

MethodError: MethodError: no method matching -(::Float64, ::Array{Float64,2})
Closest candidates are:
  -(::Float64, !Matched::Float64) at float.jl:397
  -(::Float64) at float.jl:387
  -(::Real, !Matched::Complex{Bool}) at complex.jl:295
  ...

In [9]:
m = 1
nobs = 1
Sm = Sm(zeros(m, nobs), zeros(m, nobs+1),zeros(m,m, nobs), zeros(m,m, nobs+1), 0, zeros(m,m))

ErrorException: invalid redefinition of constant Sm

In [28]:
X = convert(Matrix{Union{Missing,Float64}},X[2:end,])

724×120 Array{Union{Missing, Float64},2}:
  0.0012526     0.0010653     0.00767048   …   0.0148145     0.00144012 
  0.0037502     0.00486275    0.00671482        missing       missing   
  0.00381723    0.00450843   -0.0062556         missing       missing   
  0.00314537    0.00411852    0.00930512      -0.00678716   -0.000394184
  0.000427094   0.000511156   0.000966653       missing       missing   
 -0.00326644   -0.0033188    -0.00604413   …    missing       missing   
 -0.00837617   -0.00822697    0.00331371      -0.00465258   -0.00202109 
 -0.00189259   -0.00255374    0.007322          missing       missing   
 -0.00145783   -0.00131382   -0.00950251        missing       missing   
  0.00491526    0.00417518   -0.00312461       0.0147253     0.00167232 
  0.00751591    0.00884201   -0.00379359   …    missing       missing   
  0.00051197    0.00213642    0.000104457       missing       missing   
 -0.00153613   -0.00159242    0.00164789      -0.0129074     0.000754914
  ⋮      

In [43]:
innov

UndefVarError: UndefVarError: innov not defined

search: inv invoke invmod invcov invperm invsqrtm invscale invsqrtm! invlogcdf



```
inv(x)
```

Return the multiplicative inverse of `x`, such that `x*inv(x)` or `inv(x)*x` yields [`one(x)`](@ref) (the multiplicative identity) up to roundoff errors.

If `x` is a number, this is essentially the same as `one(x)/x`, but for some types `inv(x)` may be slightly more efficient.

# Examples

```jldoctest
julia> inv(2)
0.5

julia> inv(1 + 2im)
0.2 - 0.4im

julia> inv(1 + 2im) * (1 + 2im)
1.0 + 0.0im

julia> inv(2//3)
3//2
```

---

```
inv(M)
```

Matrix inverse. Computes matrix `N` such that `M * N = I`, where `I` is the identity matrix. Computed by solving the left-division `N = M \ I`.

# Examples

```jldoctest
julia> M = [2 5; 1 3]
2×2 Array{Int64,2}:
 2  5
 1  3

julia> N = inv(M)
2×2 Array{Float64,2}:
  3.0  -5.0
 -1.0   2.0

julia> M*N == N*M == Matrix(I, 2, 2)
true
```


In [70]:
a = [1 2;3 4]

2×2 Array{Int64,2}:
 1  2
 3  4

In [83]:
Matrix(I,2,2)/a

2×2 Array{Float64,2}:
 -2.0   1.0
  1.5  -0.5